# Plotting Alpha and Beta

$$
\alpha = AC_{ratio} / \epsilon_{ratio}(Saturation)
$$

$$
\beta = AC / \epsilon
$$

In [15]:
from typing import cast
import pandas as pd
import numpy as np
from tfo_label_balance.misc import compute_filtered_beta, compute_filtered_alpha, compute_epsilon, compute_epsilon_ratio

data = pd.read_csv('../data/combined_LLPSA2.csv')
all_experiments = data['experiment_id'].unique()
all_experiments.sort()
dc_columns = [col for col in data.columns if col.startswith('DC_')]
ac_columns = [col for col in data.columns if col.startswith('Amp_')]

# Compute Amp (AC) ratio
ac_count = len(ac_columns)
for i in range(ac_count // 2):
    data[f"AC Ratio_{i + 1}"] = data[ac_columns[i]].to_numpy() / data[ac_columns[i + ac_count // 2]].to_numpy()
ac_ratio_columns = [col for col in data.columns if col.startswith('AC Ratio_')]

grouping_column = 'experiment_id'

# Compute filtered beta for each experiment separately
beta_keep_threshold = 0.5
wavelengths = [735.0] * 5 + [850.0] * 5
all_experiments = data['experiment_id'].unique()
all_experiments.sort()

for wavelength, ac_column in zip(wavelengths, ac_columns):
    # Initialize beta column with NaN
    data[f"Beta_{ac_column}"] = np.nan
    data[f"Predicted_{ac_column}"] = np.nan
    data[f"Epsilon_{ac_column}"] = np.nan
    # Compute beta for each experiment separately
    for experiment_id in all_experiments:
        # Get data for this experiment
        experiment_mask = data['experiment_id'] == experiment_id
        ac_channel = cast(pd.Series, data.loc[experiment_mask, ac_column])
        ac_channel_np = ac_channel.to_numpy()
        
        saturation = cast(pd.Series, data.loc[experiment_mask, 'fSaO2'])
        saturation_np = saturation.to_numpy() / 100.0
        # Compute filtered beta for this experiment
        filtred_beta, keep_indices = compute_filtered_beta(ac_channel_np, saturation_np, wavelength, beta_keep_threshold)
        epsilon = compute_epsilon(saturation_np, wavelength)
        
        # Assign beta value to all rows in this experiment
        data.loc[experiment_mask, f"Beta_{ac_column}"] = filtred_beta
        data.loc[experiment_mask, f"Predicted_{ac_column}"] = epsilon * filtred_beta
        data.loc[experiment_mask, f"Epsilon_{ac_column}"] = epsilon

beta_columns = [col for col in data.columns if col.startswith('Beta_')]
predicted_ac_columns = [col for col in data.columns if col.startswith('Predicted_Amp_')]
epsilon_columns = [col for col in data.columns if col.startswith('Epsilon_Amp_')]

# Comput filtered alpha for each experiment separately
alpha_keep_threshold = 0.5
for ac_ratio_column in ac_ratio_columns:
    # Initialize alpha column with NaN
    data[f"Alpha_{ac_ratio_column}"] = np.nan
    data[f"Predicted_{ac_ratio_column}"] = np.nan
    data[f"Epsilon Ratio_{ac_ratio_column}"] = np.nan
    # Compute alpha for each experiment separately
    for experiment_id in all_experiments:
        # Get data for this experiment
        experiment_mask = data['experiment_id'] == experiment_id
        ac_ratio = cast(pd.Series, data.loc[experiment_mask, ac_ratio_column])
        ac_ratio_np = ac_ratio.to_numpy()
        saturation = cast(pd.Series, data.loc[experiment_mask, 'fSaO2'])
        saturation_np = saturation.to_numpy() / 100.0
        # Compute filtered alpha for this experiment
        filtred_alpha, keep_indices = compute_filtered_alpha(ac_ratio_np, saturation_np, alpha_keep_threshold)
        epsilon_ratio = compute_epsilon_ratio(saturation_np, 735.0, 850.0)
        
        # Assign alpha value to all rows in this experiment
        data.loc[experiment_mask, f"Alpha_{ac_ratio_column}"] = filtred_alpha
        data.loc[experiment_mask, f"Predicted_{ac_ratio_column}"] = epsilon_ratio * filtred_alpha
        data.loc[experiment_mask, f"Epsilon Ratio_{ac_ratio_column}"] = epsilon_ratio
alpha_columns = [col for col in data.columns if col.startswith('Alpha_')]
predicted_ac_ratio_columns = [col for col in data.columns if col.startswith('Predicted_AC Ratio_')]
epsilon_ratio_columns = [col for col in data.columns if col.startswith('Epsilon Ratio_')]

In [ ]:
from ipywidgets import interact, Dropdown
import matplotlib.pyplot as plt

# Define colors for each plot type
plot_colors = {
    'alpha': 'blue',
    'beta': 'red',
    'AC ratio': 'green',
    'AC': 'purple',
    'Predicted AC': 'cyan',
    'Predicted AC ratio': 'magenta',
    'Epsilon' : 'brown',
    'Epsilon Ratio': 'gray',
    'Saturation': 'orange',
}
all_plot_types = plot_colors.keys()

def plot_10_subplots(exp_data, columns, color, ylabel, axes):
    """Plot data across 10 subplots (first 5 on left column, next 5 on right column)."""
    for i, col in enumerate(columns):
        ax = axes[i % 5, i // 5]
        ax.set_visible(True)
        ax.plot(exp_data[col].values, color=color)
        ax.set_title(col)
        ax.set_xlabel('Index')
        ax.set_ylabel(ylabel)
        ax.grid(True)

def plot_5_subplots(exp_data, columns, color, ylabel, axes):
    """Plot data across 5 subplots (left column only)."""
    for i, col in enumerate(columns):
        ax = axes[i, 0]
        ax.set_visible(True)
        ax.plot(exp_data[col].values, color=color)
        ax.set_title(col)
        ax.set_xlabel('Index')
        ax.set_ylabel(ylabel)
        ax.grid(True)

def plot_data(experiment_id, plot_type):
    assert plot_type in all_plot_types, f"Invalid plot type: {plot_type}"
    # Filter data for selected experiment
    exp_data = data[data['experiment_id'] == experiment_id]
    
    # Create figure with 6 rows and 2 columns
    fig, axes = plt.subplots(6, 2, figsize=(10, 12))
    fig.suptitle(f'Experiment: {experiment_id} - Plot Type: {plot_type}', fontsize=16)
    
    # Flatten axes for easier indexing
    axes_flat = axes.flatten()
    
    # Hide all axes initially
    for ax in axes_flat:
        ax.set_visible(False)
    
    if plot_type == 'alpha':
        plot_5_subplots(exp_data, alpha_columns, plot_colors['alpha'], 'Alpha', axes)
    
    elif plot_type == 'beta':
        plot_10_subplots(exp_data, beta_columns, plot_colors['beta'], 'Beta', axes)
    
    elif plot_type == 'AC ratio':
        plot_5_subplots(exp_data, ac_ratio_columns, plot_colors['AC ratio'], 'AC Ratio', axes)
    
    elif plot_type == 'AC':
        plot_10_subplots(exp_data, ac_columns, plot_colors['AC'], 'AC (Amp)', axes)
    
    elif plot_type == 'Predicted AC':
        plot_10_subplots(exp_data, predicted_ac_columns, plot_colors['Predicted AC'], 'Predicted AC (Amp)', axes)
    
    elif plot_type == 'Predicted AC ratio':
        plot_5_subplots(exp_data, predicted_ac_ratio_columns, plot_colors['Predicted AC ratio'], 'Predicted AC Ratio', axes)
    
    elif plot_type == 'Epsilon':
        plot_10_subplots(exp_data, epsilon_columns, plot_colors['Epsilon'], 'Epsilon', axes)
    
    elif plot_type == 'Epsilon Ratio':
        plot_5_subplots(exp_data, epsilon_ratio_columns, plot_colors['Epsilon Ratio'], 'Epsilon Ratio', axes)
    
    else:
        raise NotImplementedError(f"Plot type '{plot_type}' not implemented.")
    
    # Always plot saturation on bottom left
    ax_sat = axes[5, 0]
    ax_sat.set_visible(True)
    ax_sat.plot(exp_data['fSaO2'].values, color=plot_colors['Saturation'])
    ax_sat.set_title('Saturation (fSaO2)')
    ax_sat.set_xlabel('Index')
    ax_sat.set_ylabel('Saturation (%)')
    ax_sat.grid(True)
    
    plt.tight_layout()
    plt.show()

# Create interactive widget
interact(plot_data, 
         experiment_id=Dropdown(options=all_experiments.tolist(), description='Experiment:'),
         plot_type=Dropdown(options=all_plot_types, description='Plot Type:'))

interactive(children=(Dropdown(description='Experiment:', options=('sp2021_S1R2', 'sp2021_S1R3', 'sp2021_S2R1'…

<function __main__.plot_data(experiment_id, plot_type)>